# About notebook

This is pretty much a clone of https://www.kaggle.com/code/jhoward/getting-started-with-nlp-for-absolute-beginners. But the original notebook cannot be submitted for the competiotion because it does pip install and also downloads some pretrained model - ie. needs the internet. I took some learnings from https://www.kaggle.com/code/miwojc/getting-started-with-nlp-offline-version/notebook and was able to modify the notebook and get to a submittable notebook


# Inputs
1) competition data: If i tag the notebook to the competition, kaggle automatically downloads the input,out data to the environment where the notebook runs
2) pre trained model: I randomly found someone created a dataset for debertav3small - pretrained model. Later I have more details on how the model is used




Load pretrained debertav3small model and create a tokenizer with it. 


In [1]:
import os
# I get the following annoying warning all the time
# huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
# To disable this warning, you can either:
#  - Avoid using `tokenizers` before the fork if possible
#  - Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [2]:
import numpy as np, matplotlib.pyplot as plt

from transformers import AutoModelForSequenceClassification,AutoTokenizer
# model_nm = 'microsoft/deberta-v3-small'
# model_nm = '../input/debertav3small/checkpoint'
model_nm = '../input/deberta-v3-large'
tokz = AutoTokenizer.from_pretrained(model_nm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


tokz.tokenize(string)
* tokenizes the sentence into words
* beginining of each word is prefixed by ▁ in the tokenized word
* unknown words are broken into smaller parts. See the output of the below snippet

tokx(string)
* tokenizes the sentence into words and returns a vector of integer ids that can be used for nlp training/inference

In [3]:
print(tokz.tokenize("today's a nice day"))
print(tokz("today's a nice day"))

['▁today', "'", 's', '▁a', '▁nice', '▁day']
{'input_ids': [1, 561, 280, 268, 266, 1085, 406, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


1. Import the kaggle training, eval datasets as pandas


In [4]:
from pathlib import Path
import pandas as pd

dataset_path = Path("/kaggle/input/us-patent-phrase-to-phrase-matching")
full_train_df = pd.read_csv(f'{dataset_path}/train.csv')
eval_df = pd.read_csv(dataset_path/'test.csv')

In [5]:
full_train_df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [6]:
full_train_df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


Debug: What is `include=` in describe
* what all types of objects should the summary be described for
* default is numeric types

In [7]:
full_train_df.describe(include='all')

,id,anchor,target,context,score
count,36473,36473,36473,36473,36473.000000
unique,36473,733,29340,106,NaN
top,37d61fd2272659b1,component composite coating,composition,H01,NaN
freq,1,152,24,2186,NaN
mean,NaN,NaN,NaN,NaN,0.362062
std,NaN,NaN,NaN,NaN,0.258335
min,NaN,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,NaN,0.250000
50%,NaN,NaN,NaN,NaN,0.250000
75%,NaN,NaN,NaN,NaN,0.500000


Tokenization and creating the input
* we create a new input column that is `"TEXT1: <context>; TEXT2: <target>; ANC1: <anchor>
* we then convert to a Dataset type and tokenize this to numbers
* TODO: what is the impact of calling the placeholders TEXT1, TEXT2? The anchor is an opaque string that indicates the patent subject. Would it make sense to leverage it somehow? Why/Why not?

datasets.Dataset is the rage
* It seems the preferred way to deal with "datasets" is this library called datasets. 
* We convert the dataframes into Dataset type.
* We rename the output label from score to label because thats what the transformers library's Trainer class likes.
* Then also create a train-test split. Apparently random is not great. Quoting the original notebook from which this is taken
> In practice, a random split like we've used here might not be a good idea -- here's what Dr Rachel Thomas has to say about it:
>
> "One of the most likely culprits for this disconnect between results in development vs results in production is a poorly chosen validation set (or even worse, no validation set at all). Depending on the nature of your data, choosing a validation set can be the most important step. Although sklearn offers a train_test_split method, this method takes a random subset of the data, which is a poor choice for many real-world problems."
>
> I strongly recommend reading her article [How (and why) to create a good validation set](https://www.fast.ai/2017/11/13/validation-sets/) to more fully understand this critical topic.


In [8]:
from datasets import Dataset,DatasetDict

def create_ds(df):
    def tok_func(x): return tokz(x["input"])
    df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor
    ds = Dataset.from_pandas(df).map(tok_func, batched=True)
    if 'score' in df.columns:
        # transformers library wants the output label to be label and not score
        # score column is there only for the train set and not eval set. So do a check before
        # the rename
        ds = ds.rename_column('score', 'label')
    return ds


full_train_ds = create_ds(full_train_df)
eval_ds = create_ds(eval_df)
train_test_split = full_train_ds.train_test_split(0.25, seed=42)


  0%|          | 0/37 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Correlation: While training or while submission the we need to know the accuracy. define a correlation function to pass to the model the function that emits the accuracy of the predictions

In [9]:
def corr(x,y): 
    return np.corrcoef(x,y)[0][1]

def corr_d(eval_pred): 
    return {'pearson': corr(*eval_pred)}

In [10]:
from transformers import TrainingArguments,Trainer

# TODO: Learn how to do hyperparam search/tuning


# bs=128: We do gradient descent on 128 samples at a time.
# epochs=4: The model is trained with the full dataset 4 times
# So if we have 1024 samples, we do gradient descent for 128 samples at a time
# and 4 times => 32 gradient descents
bs = 32
epochs = 4 # TODO: should this be 8? What is the right number? 
# TODO: I dont know what is a good value here for learning rate. What is too small? What is too large?
lr = 8e-5

args = TrainingArguments(
    'outputs', 
    learning_rate=lr, 
    warmup_ratio=0.1, # TODO: why?
    # IIUC to avoid overfitting we cut down the learning rate using some cosine formula. 
    # learning_rate(epoch_number) = (lr/2)(1+cos(pi*epoch_number/total_epochs)). So the 
    # learning rate keeps getting smaller
    # TODO: why all this cosine stuff. Why not lr*(1-(epoch_number/total_epochs))?
    lr_scheduler_type='cosine', # TODO: why?
    fp16=True,
    # model is evaluated after each epoch. This makes more sense if we have hyper param tuning i think
    # E.g we can have some settings to early stop the training if its degrading
    evaluation_strategy="epoch",
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs*2, # TODO: why?
    num_train_epochs=epochs,# TODO: why?
    # TODO: why? I vaguely remember that larger weights imply that models are more "decisive"
    # and often predict strong values E.g sigmoid function output is between 0 to 1. large weights result
    # in sigmoid(f(X)) being very close to 0 or very close to 1. IOW: likely change of overfitting
    # So "artificially" make the weights smaller as we train with more and more data.
    # TODO: I think the dataset is pretty small "here" and we have "only" 4 epochs. So is that relevant?
    weight_decay=0.01,
    report_to='none'
)

Load the sequence classification model and train

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)

trainer = Trainer(
    model, 
    args, 
    train_dataset=train_test_split['train'], 
    eval_dataset=train_test_split['test'],
    tokenizer=tokz, 
    compute_metrics=corr_d
)

Some weights of the model checkpoint at ../input/deberta-v3-large were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a

In [12]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.026888,0.792787
2,0.043600,0.024938,0.830176
3,0.018800,0.020369,0.845814
4,0.010300,0.020019,0.847892


/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=1712, training_loss=0.022219586664828185, metrics={'train_runtime': 1732.1829, 'train_samples_per_second': 63.167, 'train_steps_per_second': 0.988, 'total_flos': 4841956735068972.0, 'train_loss': 0.022219586664828185, 'epoch': 4.0})

In [13]:
preds = trainer.predict(eval_ds).predictions.astype(float)

preds = np.clip(preds, 0, 1) # Outputs are not 0-1. So clip them
eval_df.head()

,id,anchor,target,context,input
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,TEXT1: G02; TEXT2: inorganic photoconductor dr...
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,TEXT1: F23; TEXT2: altering gas flow; ANC1: ad...
2,36baf228038e314b,lower trunnion,lower locating,B60,TEXT1: B60; TEXT2: lower locating; ANC1: lower...
3,1f37ead645e7f0c8,cap component,upper portion,D06,TEXT1: D06; TEXT2: upper portion; ANC1: cap co...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,TEXT1: H04; TEXT2: artificial neural network; ...


In [14]:
print(preds[:10])
print(preds[:,0][:10])

[[0.54541016]
 [0.61181641]
 [0.49731445]
 [0.20507812]
 [0.        ]
 [0.51025391]
 [0.50878906]
 [0.        ]
 [0.25585938]
 [1.        ]]
[0.54541016 0.61181641 0.49731445 0.20507812 0.         0.51025391
 0.50878906 0.         0.25585938 1.        ]


In [15]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds[:,0]
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1031

In [16]:
%%bash 
head submission.csv

id,score
4112d61851461f60,0.54541015625
09e418c93a776564,0.61181640625
36baf228038e314b,0.497314453125
1f37ead645e7f0c8,0.205078125
71a5b6ad068d531f,0.0
474c874d0c07bd21,0.51025390625
442c114ed5c4e3c9,0.5087890625
b8ae62ea5e1d8bdb,0.0
faaddaf8fcba8a3f,0.255859375
